# <font color=4169E1> Watchis
---
**versão: 1.0.5**
## Intro:

  
Watchis é um sistema com uso de machine-learning para recomendação de filmes. Vamos utilizar os seguintes datasets:
<br>
The Movies Datasets: <a href="https://www.kaggle.com/rounakbanik/the-movies-dataset">Link para o dataset</a>
<br>
TMDB_5000: <a href="https://www.kaggle.com/tmdb/tmdb-movie-metadata">Link para o dataset</a>


## Sumário

###1.Análise exploratória dos dados
###2.Introdução ao Machine Learning

**Documentação:** <a href="https://docs.google.com/document/d/1FQ8ZlsU1m7-uFC_SU_d6jtJEfITCK8wDmtYTDA20Z9o/edit">Artigo Watchis</a>

##<font color=4169E1>Importação das Bibliotecas e Datasets

In [ ]:
!pip install seaborn==0.11.0 &> /dev/null
!pip install -U pandas-profiling &> /dev/null
!pip install scikit-surprise &> /dev/null
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer #TF-IDF
from sklearn.feature_extraction.text import CountVectorizer #Count
from sklearn.metrics.pairwise import linear_kernel #Similaridade
from sklearn.metrics.pairwise import cosine_similarity #Similaridade
from surprise import Reader, Dataset, SVD
from surprise.accuracy import rmse, mae
from surprise.model_selection import cross_validate
from collections import defaultdict


from datetime import datetime
from pandas_profiling import ProfileReport
sns.set_theme(style="whitegrid")

In [ ]:
notas = pd.read_csv('https://raw.githubusercontent.com/Guilherme-Farias/Watchis-backend/datasets/ratings_tmdb_small.csv', low_memory=False)
filmes = pd.read_csv('https://raw.githubusercontent.com/Guilherme-Farias/Watchis-backend/master/tmdb_5000_movies.csv', low_memory=False)
creditos = pd.read_csv('https://raw.githubusercontent.com/Guilherme-Farias/Watchis-backend/master/tmdb_5000_credits.csv', low_memory=False)

## <font color=4169E1>Pré-processamento de dados

In [ ]:
#Traduzindo as colunas
notas.columns = ["usuarioId", "filmeId", "nota"]
filmes.columns = ['orcamento', 'generos', 'pagina_inicial', 'filmeId', 'palavras_chave', 'idioma_original', 'titulo_original', 'visao_geral', 'popularidade', 'companhias_producao', 'paises_producao', 'data_lancamento', 'receita', 'tempo_de_filme', 'lingua_falada', 'status', 'slogan', 'titulo', 'media_notas', 'total_notas']
creditos.columns = ['filmeId', 'titulo', 'elenco', 'equipe']

#Transformando ID's em int
filmes['filmeId'] = filmes['filmeId'].astype('int')
creditos['filmeId'] = creditos['filmeId'].astype('int')
notas['filmeId'] = notas['filmeId'].astype('int')

#Unindo Datasets
filmes = filmes.merge(creditos[['filmeId','elenco','equipe']], on='filmeId')

#Tirando colunas que não vamos utilizar
filmes.drop(labels = ['pagina_inicial'], axis='columns', inplace=True)
filmes.titulo.drop_duplicates(inplace=True)

# Colocando as colunas em uma ordem mais sintática
filmes = filmes.reindex(columns = ['filmeId', 'titulo','titulo_original','data_lancamento','visao_geral',
                                   'slogan','generos', 'palavras_chave', 'tempo_de_filme', 'idioma_original','lingua_falada','companhias_producao',
                                   'paises_producao', 'orcamento', 'receita','status',  'media_notas','total_notas','popularidade', 
                                   'elenco', 'equipe'])
# Transformando tipos
filmes['receita'] = filmes['receita'].astype(int)
filmes['orcamento'] = filmes['orcamento'].astype(int)
filmes['popularidade'] = filmes['popularidade'].astype(int)
filmes['tempo_de_filme'] = pd.to_timedelta(filmes['tempo_de_filme'], unit='m')
filmes['data_lancamento'] = pd.to_datetime(filmes['data_lancamento'], format='%Y/%m/%d', errors='coerce')

In [ ]:
# Pega o diretor da lista de equipe
def get_diretor(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

# Colunas que iremos transformar de string de Json, para dicionario de python e depois para lista do python
colunas = ['elenco', 'generos', 'lingua_falada', 'paises_producao', 'companhias_producao', 'palavras_chave']

for coluna in colunas:
  filmes[coluna] = filmes[coluna].fillna('[]').apply(literal_eval)
  filmes[coluna] = filmes[coluna].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])


# Tratando dados do diretor
filmes['equipe'] = filmes['equipe'].fillna('[]').apply(literal_eval)
filmes['diretor'] = filmes['equipe'].apply(get_diretor)
filmes['diretor'].fillna('',inplace = True)
filmes.drop(['equipe'],axis = 1,inplace = True)

In [ ]:
#Trocando NaN com uma String vazia
filmes['visao_geral'] = filmes['visao_geral'].fillna('')
filmes['slogan'] = filmes['slogan'].fillna('')

##<font color=4169E1> Pandas Profiling nos dados de filmes

In [ ]:
# profile = ProfileReport(filmes, title='Profiling Movies',html={'style':{'full_width':True}})
# profile.to_notebook_iframe()

In [ ]:
# profile = ProfileReport(notas, title='Profiling Ratings',html={'style':{'full_width':True}})
# profile.to_notebook_iframe()

# <font color=4169E1> Avaliação ponderada

## <font color=4169E1> Função

In [ ]:
# Pegando variaveis para utilizar na próxima fórmula
C = filmes.media_notas.mean()
m = filmes.total_notas.quantile(0.95)
print(f'Média das notas médias(C): {C}\nMínimo de votos(m): {m}')

Média das notas médias(C): 6.092171559442011
Mínimo de votos(m): 3040.8999999999996


In [ ]:
# Baseado na formula do IMDB
def avaliacao_ponderada(df, m=m, c=C):
  v = df['total_notas']
  R = df['media_notas']
  return (v/(v+m) * R) + (m/(m+v) * C)

## <font color=4169E1> Recomendação

In [ ]:
# Pegando os filmes com uma quantidade minima de votos
filmes_com_mais_de_m_votos = filmes.copy().loc[filmes['total_notas'] >= m]
filmes_com_mais_de_m_votos['avaliacao_ponderada'] = filmes_com_mais_de_m_votos.apply(avaliacao_ponderada, axis=1);
filmes_com_mais_de_m_votos = filmes_com_mais_de_m_votos.sort_values('avaliacao_ponderada', ascending=False)
filmes_com_mais_de_m_votos[['titulo', 'total_notas', 'media_notas', 'avaliacao_ponderada']].head(15)

,titulo,total_notas,media_notas,avaliacao_ponderada
1881,The Shawshank Redemption,8205,8.5,7.848921
65,The Dark Knight,12002,8.2,7.773906
662,Fight Club,9413,8.3,7.760909
96,Inception,13752,8.1,7.736417
3232,Pulp Fiction,8428,8.3,7.714609
95,Interstellar,10867,8.1,7.660997
809,Forrest Gump,7927,8.2,7.615595
3337,The Godfather,5893,8.4,7.614467
329,The Lord of the Rings: The Return of the King,8064,8.1,7.550188
262,The Lord of the Rings: The Fellowship of the Ring,8705,8.0,7.506082


# <font color=4169E1> Baseado em conteudo

In [ ]:
# Pegando os indices dos filmes
indices = pd.Series(filmes.index, index=filmes['titulo']).drop_duplicates()

## <font color=4169E1> Função

### <font color=4169E1>  CBF sem verificação

In [ ]:
def normal_CBF(titulo, similaridade):
  # Pega o Index do filme pelo título(Index != Id)
  idx = indices[titulo]

  # Pega a similaridade de acordo com esse Id nessa matrix de similaridades
  sim_scores = list(enumerate(similaridade[idx]))

  # Copia a base de filmes, para por um campo de similaridade para esse filme especifico
  filmes_similares = filmes.copy()
  filmes_similares['grau_de_similaridade'] = pd.Series(sim_scores).apply(lambda x: x[1])

  # Ordena os os filmes de maior similaridade à menor similaridade
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  # Pega os 10 filmes mais similares(tirando o primeiro, pois o primeiro é ele)
  sim_scores = sim_scores[1:11]

  # Pega o Index dos filmes
  filme_indices = [i[0] for i in sim_scores]

  # Retorna os dez filmes mais similares
  return filmes_similares[['titulo', 'data_lancamento','total_notas', 'media_notas', 'grau_de_similaridade']].iloc[filme_indices]

### <font color=4169E1> CFB verificando os melhores filmes

In [ ]:
def avaliacao_ponderada_CBF(titulo, similaridade):
  # Pega o Index do filme pelo título(Index != Id)
  idx = indices[titulo]

  # Pega a similaridade de acordo com esse Id nessa matrix de similaridades
  sim_scores = list(enumerate(similaridade[idx]))

  # Copia a base de filmes, para por um campo de similaridade para esse filme especifico
  filmes_similares = filmes.copy()
  filmes_similares['grau_de_similaridade'] = pd.Series(sim_scores).apply(lambda x: x[1])

  # Ordena os os filmes de maior similaridade à menor similaridade
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  # Pega os 100 filmes mais similares(tirando o primeiro, pois o primeiro é ele)
  sim_scores = sim_scores[1:101]

  # Pega o Index dos filmes
  filme_indices = [i[0] for i in sim_scores]

  # Pega os dados inteiros dos 100 filmes mais similares
  filmes_similares = filmes_similares.iloc[filme_indices][['titulo', 'total_notas', 'media_notas', 'data_lancamento']]

  # Prepara os dados para avaliação ponderada
  C = filmes_similares.media_notas.mean()
  m = filmes_similares.total_notas.quantile(0.60)

  # Faz a valiação ponderada com os 100 filmes mais similares
  filmes_similares = filmes_similares.loc[filmes_similares['total_notas'] >= m]
  filmes_similares['avaliacao_ponderada'] = filmes_similares.apply(avaliacao_ponderada, axis=1);

  # Retorna os dez filmes mais similares de acordo com sua avaliação ponderada
  return filmes_similares.sort_values('avaliacao_ponderada', ascending=False).head(15)

### <font color=4169E1> CFB verificando a similaridade com a quantidade de notas

In [ ]:
def similaridade_ponderada(df, m=m, C=C):
  v = df['total_notas']
  R = df['grau_de_similaridade']
  return (v/(v+m) * R) + (m/(m+v) * C)

In [ ]:
def similaridade_ponderada_CBF(titulo, similaridade):
  # Pega o Index do filme pelo título(Index != Id)
  idx = indices[titulo]

  # Pega a similaridade de acordo com esse Id nessa matrix de similaridades
  sim_scores = list(enumerate(similaridade[idx]))

  # Copia a base de filmes, para por um campo de similaridade para esse filme especifico
  filmes_similares = filmes.copy()
  filmes_similares['grau_de_similaridade'] = pd.Series(sim_scores).apply(lambda x: x[1])
  filmes_similares = filmes_similares[['titulo', 'total_notas', 'media_notas', 'grau_de_similaridade']]
  
  # Preparando valores para uma similaridade ponderada envolvendo o total de votos de usuário
  m = filmes_similares.total_notas.quantile(0.10)
  C = filmes_similares.grau_de_similaridade.mean()

  # Faz a similaridade ponderada
  filmes_similares['similaridade_ponderada'] = filmes_similares.apply(similaridade_ponderada, args=(m, C), axis=1)


  # Retorna os dez filmes mais similares de acordo com essa "similaridade ponderada"
  return filmes_similares.sort_values('similaridade_ponderada', ascending=False)[1:11]

## <font color=4169E1> Similaridades

### <font color=4169E1> Baseado em conteudo focado na visão geral

In [ ]:
#TF-IDF Tira todas as palavras em inglês que mais se repetem
tfidf = TfidfVectorizer(stop_words='english')

# Constroi a Matrix 
tfidf_matrix = tfidf.fit_transform(filmes['visao_geral'])

# Mostra o tamanho da Matrix
tfidf_matrix.shape

(4803, 20978)

In [ ]:
# Verifica a similaridade dentro da Matrix
similaridade_1 = linear_kernel(tfidf_matrix, tfidf_matrix)

#### <font color=4169E1> Exemplo 1

In [ ]:
normal_CBF('The Dark Knight', similaridade_1)

,titulo,data_lancamento,total_notas,media_notas,grau_de_similaridade
3,The Dark Knight Rises,2012-07-16,9106,7.6,0.301512
428,Batman Returns,1992-06-19,1673,6.6,0.249431
3854,"Batman: The Dark Knight Returns, Part 2",2013-01-18,419,7.9,0.224516
299,Batman Forever,1995-05-31,1498,5.2,0.214070
1359,Batman,1989-06-23,2096,7.0,0.182334
119,Batman Begins,2005-06-10,7359,7.5,0.162037
1181,JFK,1991-12-20,502,7.5,0.134627
9,Batman v Superman: Dawn of Justice,2016-03-23,7004,5.7,0.118637
2507,Slow Burn,2005-09-12,16,5.5,0.113738
210,Batman & Robin,1997-06-20,1418,4.2,0.106896


#### <font color=4169E1> Exemplo 2

In [ ]:
avaliacao_ponderada_CBF('The Dark Knight', similaridade_1)

,titulo,total_notas,media_notas,data_lancamento,avaliacao_ponderada
571,Inglourious Basterds,6430,7.9,2009-08-18,7.319546
3,The Dark Knight Rises,9106,7.6,2012-07-16,7.222525
351,The Departed,4339,7.9,2006-10-05,7.155081
119,Batman Begins,7359,7.5,2005-06-10,7.088355
82,Dawn of the Planet of the Apes,4410,7.3,2014-06-26,6.807055
506,Despicable Me 2,4637,7.0,2013-06-25,6.640447
205,Sherlock Holmes: A Game of Shadows,3886,7.0,2011-11-22,6.601465
531,The Man from U.N.C.L.E.,2265,7.1,2015-08-13,6.522397
2585,The Hurt Locker,1840,7.2,2008-10-10,6.509800
2536,The Deer Hunter,921,7.8,1978-12-08,6.489181


#### <font color=4169E1> Exemplo 3

In [ ]:
similaridade_ponderada_CBF('The Dark Knight', similaridade_1)

,titulo,total_notas,media_notas,grau_de_similaridade,similaridade_ponderada
3,The Dark Knight Rises,9106,7.6,0.301512,0.301122
428,Batman Returns,1673,6.6,0.249431,0.247694
3854,"Batman: The Dark Knight Returns, Part 2",419,7.9,0.224516,0.218420
299,Batman Forever,1498,5.2,0.214070,0.212413
1359,Batman,2096,7.0,0.182334,0.181328
119,Batman Begins,7359,7.5,0.162037,0.161782
1181,JFK,502,7.5,0.134627,0.131614
9,Batman v Superman: Dawn of Justice,7004,5.7,0.118637,0.118444
210,Batman & Robin,1418,4.2,0.106896,0.106046
879,Law Abiding Citizen,1486,7.2,0.103289,0.102506


### <font color=4169E1> Baseado em conteudo focado nos créditos

In [ ]:
# Faz uma cópia de filmes para 
copia_filmes = filmes.copy()

In [ ]:
# Pega as listas e transformam em uma grande String
def prepara_dados(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''
colunas = ['elenco', 'palavras_chave', 'diretor', 'generos', 'companhias_producao']
for coluna in colunas:
    copia_filmes[coluna] = copia_filmes[coluna].apply(prepara_dados)

In [ ]:
# Pega as String das colunas e transformam em uma String de todas juntas, chamada de sopa
def cria_sopa(x):
    return ' '.join(x['palavras_chave']) + ' ' + ' '.join(x['elenco']) + ' ' + x['diretor'] + ' ' + ' '.join(x['generos'])
copia_filmes['sopa'] = copia_filmes.apply(cria_sopa, axis=1)

In [ ]:
# CountVectorizer Tira todas as palavras em inglês que mais se repetem
count = CountVectorizer(stop_words='english')

# Constroi a Matrix 
count_matrix = count.fit_transform(copia_filmes['sopa'])

# Mostra o tamanho da Matrix
count_matrix.shape

(4803, 66904)

In [ ]:
similaridade_2 = cosine_similarity(count_matrix, count_matrix)

#### <font color=4169E1> Exemplo 1

In [ ]:
normal_CBF('The Dark Knight', similaridade_2)

,titulo,data_lancamento,total_notas,media_notas,grau_de_similaridade
4638,Amidst the Devil's Wings,2014-12-01,0,0.0,0.136505
119,Batman Begins,2005-06-10,7359,7.5,0.135133
3,The Dark Knight Rises,2012-07-16,9106,7.6,0.120386
428,Batman Returns,1992-06-19,1673,6.6,0.109754
210,Batman & Robin,1997-06-20,1418,4.2,0.102444
3972,Chicago Overcoat,2009-10-10,5,6.1,0.099689
4408,Jimmy and Judy,2006-01-01,8,5.4,0.087433
1359,Batman,1989-06-23,2096,7.0,0.084929
299,Batman Forever,1995-05-31,1498,5.2,0.084130
3644,Albino Alligator,1996-09-09,45,5.6,0.084013


#### <font color=4169E1> Exemplo 2

In [ ]:
avaliacao_ponderada_CBF('The Dark Knight', similaridade_2)

,titulo,total_notas,media_notas,data_lancamento,avaliacao_ponderada
3,The Dark Knight Rises,9106,7.6,2012-07-16,7.222525
119,Batman Begins,7359,7.5,2005-06-10,7.088355
1359,Batman,2096,7.0,1989-06-23,6.462591
428,Batman Returns,1673,6.6,1992-06-19,6.272404
2371,RockNRolla,836,6.9,2008-09-04,6.266369
919,Payback,548,6.7,1999-02-05,6.184983
1740,Kick-Ass 2,2224,6.3,2013-07-17,6.179962
1245,Colombiana,824,6.5,2011-07-27,6.179121
2915,Trash,242,7.1,2014-10-09,6.166464
3518,The Protector,167,6.8,2005-08-11,6.129020


#### <font color=4169E1> Exemplo 3

In [ ]:
similaridade_ponderada_CBF('The Dark Knight', similaridade_2)

,titulo,total_notas,media_notas,grau_de_similaridade,similaridade_ponderada
119,Batman Begins,7359,7.5,0.135133,0.134943
3,The Dark Knight Rises,9106,7.6,0.120386,0.120252
428,Batman Returns,1673,6.6,0.109754,0.109106
210,Batman & Robin,1418,4.2,0.102444,0.101742
1359,Batman,2096,7.0,0.084929,0.084553
299,Batman Forever,1498,5.2,0.084130,0.083611
2793,The Killer Inside Me,173,6.0,0.081396,0.077338
4099,Harsh Times,197,6.3,0.080436,0.076900
3819,Defendor,195,6.5,0.078811,0.075335
2915,Trash,242,7.1,0.075836,0.073143


# <font color=4169E1> Baseado em Filtro Colaborativo

## <font color=4169E1> Analisando filtros

In [ ]:
# Mostra formatado em DataFrame os dados estatisticos da frequencia de filmes e usuarios
freq_filmes = notas['filmeId'].value_counts().describe().to_frame().reset_index()
freq_usuarios = notas['usuarioId'].value_counts().describe().to_frame().reset_index()
df_formatado = freq_filmes.merge(freq_usuarios[['index','usuarioId']], on='index')
df_formatado = df_formatado.set_index('index')
df_formatado.columns = ['frequencia_filme', 'frequencia_usuario']
df_formatado

,frequencia_filme,frequencia_usuario
index,,
count,3535.000000,610.000000
mean,19.856860,115.072131
std,32.339699,169.451180
min,1.000000,8.000000
25%,3.000000,27.000000
50%,7.000000,53.000000
75%,22.000000,129.750000
max,329.000000,1605.000000


In [ ]:
notas.filmeId.value_counts().quantile(0.80)

30.0

In [ ]:
# Pega a quantidade minima de votos que um filme tem que ter para ser contabilizado no nosso filtro colaborativo
filtro_filmes = notas.filmeId.value_counts() > notas.filmeId.value_counts().quantile(0.80)
filtro_filmes = filtro_filmes[filtro_filmes].index.tolist()
print(f'Quantidade de dados antes do filtro: {notas.shape}')
notas = notas[(notas['filmeId'].isin(filtro_filmes))]
print(f'Quantidade de dados antes do filtro: {notas.shape}')

Quantidade de dados antes do filtro: (70194, 3)
Quantidade de dados antes do filtro: (48401, 3)


In [ ]:
# Adicionando algumas avaliações, para analisarmos com nosso filtro colaborativo
df = pd.DataFrame([[1000, 118340, 5.0],
                   [1000, 157336, 5.0],
                   [1000, 27205, 5.0],
                   [1000, 122, 5.0],
                   [1000, 603, 5.0],
                   [1000, 550, 5.0]], columns=notas.columns)
notas = notas.append(df, ignore_index=True)

## <font color=4169E1> Treinando modelo

In [ ]:
# Informa o range das notas, nesse sistema ele vai de 0.5 até 5
reader = Reader(rating_scale = (0.5, 5))

# Cria um dataset com Reader
data = Dataset.load_from_df(notas[['usuarioId', 'filmeId', 'nota']], reader)

In [ ]:
# Criando instancia do SVD
np.random.seed(7)
svd = SVD()

# Treinando os dados
cross_validate(svd, data, measures = ['RMSE', 'MAE'], cv=5, verbose = True)
print('>> Traning Done')

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8448  0.8470  0.8425  0.8374  0.8398  0.8423  0.0034  
MAE (testset)     0.6441  0.6485  0.6423  0.6431  0.6382  0.6432  0.0033  
Fit time          2.44    2.41    2.43    2.40    2.41    2.42    0.02    
Test time         0.07    0.07    0.06    0.08    0.26    0.11    0.08    
>> Traning Done


In [ ]:
# Cria a base de testes
trainset = data.build_full_trainset()

# Pega as notas que não estão no teste e preenchem com a média
antiset = trainset.build_anti_testset()

In [ ]:
# Pega todas as previsões
predictions = svd.test(antiset)

#### <font color=4169E1> Exemplo

In [ ]:
#exemplo de saída de uma previsão
## uid = id do usuario
## iid = id do filme
## r_ui = valor da nota do usuario no filme, caso não exista é colocado a média)
## est = valor esperado que o usuário vá por
predictions[0]

Prediction(uid=1, iid=24, r_ui=3.6800669324684447, est=4.3780757240782595, details={'was_impossible': False})

In [ ]:
# Podemos prever a nota de um filme de um usuario específico
svd.predict(1000, 279)

Prediction(uid=1000, iid=279, r_ui=None, est=4.587313102737602, details={'was_impossible': False})

## <font color=4169E1> Função

In [ ]:
# Cria um DataFrame com os 'n' filmes que foram passados
def pega_top_n(predictions, n):
    top_n = defaultdict(list)
    for uid, iid, _, est, _ in predictions:
        top_n[uid].append((iid,est))
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_n[uid] = user_ratings[:n]
    return top_n

top_n = pega_top_n(predictions, 10)

In [ ]:
# Pega os filmes recomendados
def CF(usuarioId):
  top_n_df = pd.DataFrame(top_n[usuarioId])
  top_n_df.columns = ['filmeId', 'cf']
  previsoes = filmes.copy()
  previsoes = previsoes[['filmeId', 'titulo', 'total_notas', 'media_notas']]
  previsoes = previsoes.loc[previsoes.filmeId.isin(top_n_df.filmeId)]
  previsoes = previsoes.merge(top_n_df[['filmeId','cf']], on='filmeId')
  return previsoes.sort_values('cf', ascending=False)

#### <font color=4169E1> Exemplo

In [ ]:
# Filmes que o usuario mil avaliou
filmes[filmes.filmeId.isin(notas[notas.usuarioId == 1000]['filmeId'])].titulo.to_frame()

,titulo
94,Guardians of the Galaxy
95,Interstellar
96,Inception
329,The Lord of the Rings: The Return of the King
634,The Matrix
662,Fight Club


In [ ]:
# De acordo com as notas dadas pelo usuario, podemos verificar quais filmes ele daria maior nota e recomendamos esses filmes
CF(1000)

,filmeId,titulo,total_notas,media_notas,cf
3,278,The Shawshank Redemption,8205,8.5,4.939753
4,38,Eternal Sunshine of the Spotless Mind,3652,7.9,4.771556
1,1422,The Departed,4339,7.9,4.768633
0,155,The Dark Knight,12002,8.2,4.766824
7,73,American History X,3016,8.2,4.765481
2,16869,Inglourious Basterds,6430,7.9,4.756448
8,9388,Thank You for Smoking,664,7.1,4.752730
9,629,The Usual Suspects,3254,8.1,4.749035
5,947,Lawrence of Arabia,851,7.8,4.738934
6,68,Brazil,861,7.5,4.724393


# <font color=4169E1> Baseado em Filtro Híbrido

In [ ]:
def filtragem_hibrida(usuarioId, titulo):
    # Pega o Index do filme pelo título
    idx = indices[titulo]    

    # Pega a similaridade de acordo com esse Id nessa matrix de similaridades
    sim_scores = list(enumerate(similaridade_1[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:500]

    # Pega o filmeId dos filmes
    filme_indices = [i[0] for i in sim_scores]


    # Copia a base de filmes, para por um campo de similaridade para esse filme especifico
    filmes_similares = filmes.copy().iloc[filme_indices]

    # Preve a nota do usuario para os 500 filmes mais similares
    filmes_similares['est'] = filmes_similares['filmeId'].apply(lambda x: svd.predict(usuarioId, x).est)

    # Ordena os filmes pela nota prevista
    filmes_similares = filmes_similares.sort_values('est', ascending=False)
    return filmes_similares[['filmeId', 'titulo', 'total_notas', 'media_notas', 'est']].head(10)

In [ ]:
# Um usuário qualquer
filtragem_hibrida(5, 'The Dark Knight')

,filmeId,titulo,total_notas,media_notas,est
3337,238,The Godfather,5893,8.4,4.451100
3750,5925,The Great Escape,717,7.8,4.122131
2536,11778,The Deer Hunter,921,7.8,4.115243
3,49026,The Dark Knight Rises,9106,7.6,4.028920
277,36557,Casino Royale,3855,7.3,4.005795
3454,629,The Usual Suspects,3254,8.1,3.963189
2860,387,Das Boot,613,7.9,3.958818
127,76341,Mad Max: Fury Road,9427,7.2,3.950224
571,16869,Inglourious Basterds,6430,7.9,3.919358
351,1422,The Departed,4339,7.9,3.894412


In [ ]:
# O usuário criado por nós
filtragem_hibrida(1000, 'The Dark Knight')

,filmeId,titulo,total_notas,media_notas,est
351,1422,The Departed,4339,7.9,4.768633
571,16869,Inglourious Basterds,6430,7.9,4.756448
3454,629,The Usual Suspects,3254,8.1,4.749035
3841,765,Evil Dead II,742,7.6,4.617462
3337,238,The Godfather,5893,8.4,4.548022
1829,6977,No Country for Old Men,3003,7.7,4.498377
2839,10220,Rounders,439,6.9,4.483854
510,9693,Children of Men,2071,7.4,4.453172
3075,1640,Crash,1149,7.2,4.451489
287,68718,Django Unchained,10099,7.8,4.440450
